In [1]:
import sys
import os
import requests
import numpy as np
import scipy
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.client import device_lib
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras_visualizer import visualizer
from IPython.display import display, Image

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)

2025-10-16 12:42:32.949497: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-16 12:42:33.010517: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-16 12:42:33.011871: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-16 12:42:33.990775: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import sys
import requests
import importlib

def import_local_or_github(package_name, function_name=None, directory=None, giturl=None):
    # Import functions directly from github
    # Important: note that we use raw.githubusercontent.com, not github.com

    try: # to find the file locally
        if directory is not None:
            if directory not in sys.path:
                sys.path.append(directory)

        package = importlib.import_module(package_name)
        if function_name is not None:
            function = getattr(package, function_name)
            return function
        else:
            return package

    except: # get the file from github
        if giturl is None:
            giturl = 'https://raw.githubusercontent.com/florisvb/Nonlinear_and_Data_Driven_Estimation/main/Utility/' + str(package_name) + '.py'

        r = requests.get(giturl)
        print('Fetching from: ')
        print(r)

        # Store the file to the colab working directory
        with open(package_name+'.py', 'w') as f:
            f.write(r.text)
        f.close()

        # import the function we want from that file
        package = importlib.import_module(package_name)
        if function_name is not None:
            function = getattr(package , function_name)
            return function
        else:
            return package

planar_drone = import_local_or_github('planar_drone', directory='../Utility')
plot_tme = import_local_or_github('plot_utility', 'plot_tme', directory='../Utility')
generate_training_data_utility = import_local_or_github('generate_training_data_utility', directory='../Utility')
keras_ann_utility = import_local_or_github('keras_ann_utility', directory='../Utility')
keras_advanced_utility = import_local_or_github('keras_advanced_utility', directory='../Utility')

/home/caveman/PY38/lib/python3.8/site-packages/do_mpc/sysid/__init__.py:15: UserWarning: The ONNX feature is not available. Please install the full version of do-mpc to access this feature.
  warnings.warn('The ONNX feature is not available. Please install the full version of do-mpc to access this feature.')
/home/caveman/PY38/lib/python3.8/site-packages/do_mpc/opcua/__init__.py:14: UserWarning: The opcua feature is not available. Please install the full version of do-mpc to access this feature.
  warnings.warn('The opcua feature is not available. Please install the full version of do-mpc to access this feature.')


# Download, load, clean, and add noise to data as in Lesson 12.A

In [3]:
generate_training_data_utility.download_data('planar_drone_trajectories.zip')
traj_list = generate_training_data_utility.load_trajectory_data('planar_drone_trajectories')
traj_list = generate_training_data_utility.clean_trajectory_data(traj_list)
traj_list = generate_training_data_utility.add_noise_to_trajectory_data(traj_list, 0.02)

Fetching from: https://raw.githubusercontent.com/florisvb/Nonlinear_and_Data_Driven_Estimation/main/Data/planar_drone_trajectories.zip
Successfully downloaded planar_drone_trajectories.zip (29316396 bytes)
unzipping...
Number of trajectories: 
3001
Number of good trajectories: 
2924


# Build augmented data frames

In [4]:
state_names =  ['theta', 'theta_dot', 'x', 'x_dot', 'z', 'z_dot'] 
control_names = ['j1', 'j2']
output_names = ['sensor_optic_flow', 'sensor_theta', 'sensor_theta_dot', 'sensor_accel_x', 'sensor_accel_z']

In [5]:
state_offsets = [0, -1]
control_offsets = [0, -1]
output_offsets = [0]

In [7]:
traj_augment_list = []
for traj in traj_list:
    traj_augment = keras_ann_utility.collect_offset_rows(traj, states=state_names, controls=control_names, outputs=output_names, 
                        state_offsets=state_offsets, control_offsets=control_offsets, output_offsets=output_offsets)
    
    traj_augment_list.append(traj_augment)

traj_augment_all = pd.concat(traj_augment_list, ignore_index=True)

np.round(traj_augment_all, 4)

,theta_offset_0,theta_dot_offset_0,x_offset_0,x_dot_offset_0,z_offset_0,z_dot_offset_0,theta_offset_-1,theta_dot_offset_-1,x_offset_-1,x_dot_offset_-1,z_offset_-1,z_dot_offset_-1,j1_offset_0,j2_offset_0,j1_offset_-1,j2_offset_-1,sensor_optic_flow_offset_0,sensor_theta_offset_0,sensor_theta_dot_offset_0,sensor_accel_x_offset_0,sensor_accel_z_offset_0
0,-0.1418,-16.5923,-17.5827,3.1027,1.0867,2.8330,0.1430,11.3867,-17.9304,4.4794,1.0458,-1.8849,8.0492,75.7452,-11.1830,59.7404,2.8555,-0.1297,-16.5692,10.0820,65.2603
1,-0.7740,3.5839,-17.0410,7.4923,1.6235,7.8768,-0.1418,-16.5923,-17.5827,3.1027,1.0867,2.8330,-2.3801,72.3657,8.0492,75.7452,4.5616,-0.7970,3.5455,51.1484,41.3044
2,-0.7184,-2.3685,-16.0715,12.2042,2.6601,12.3902,-0.7740,3.5839,-17.0410,7.4923,1.6235,7.8768,1.4152,54.4326,-2.3801,72.3657,4.5406,-0.7157,-2.3828,36.1400,30.8971
3,-0.7619,1.1836,-14.6472,16.0368,4.0293,15.2480,-0.7184,-2.3685,-16.0715,12.2042,2.6601,12.3902,-0.8033,31.5578,1.4152,54.4326,3.9853,-0.8000,1.1889,22.2908,12.4992
4,-0.7637,-0.7643,-12.9502,18.2044,5.6204,16.5783,-0.7619,1.1836,-14.6472,16.0368,4.0293,15.2480,0.4904,10.8784,-0.8033,31.5578,3.2409,-0.7633,-0.8192,7.5193,-1.9770
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260231,0.1179,1.4377,1.4046,0.3253,18.6757,1.4888,0.2029,-3.1839,1.3560,0.4383,18.4977,1.8236,1.0029,3.2105,1.8880,6.3435,0.0008,0.0844,1.4332,-0.3440,-6.6206
260232,0.4004,3.9516,1.4050,0.2504,18.7618,0.8048,0.1179,1.4377,1.4046,0.3253,18.6757,1.4888,-0.7430,2.0298,1.0029,3.2105,-0.0138,0.3699,3.9407,-0.7364,-7.9284
260233,0.6460,2.0458,1.4515,0.1698,18.8009,-0.0297,0.4004,3.9516,1.4050,0.2504,18.7618,0.8048,-0.8198,2.3389,-0.7430,2.0298,0.0378,0.7064,2.0834,-1.4761,-7.9760
260234,0.7855,0.0193,1.4396,-0.0029,18.8282,-0.8096,0.6460,2.0458,1.4515,0.1698,18.8009,-0.0297,0.0282,2.9830,-0.8198,2.3389,-0.0253,0.7517,-0.0115,-2.1263,-7.6949


In [8]:
n_state = len(state_names)
n_control = len(control_names)
n_output = len(output_names)
delay_embedding = len(state_offsets)

print('States:', n_state)
print('Controls:', n_control)
print('Outputs:', n_output)
print('Delay embedding:', delay_embedding)

States: 6
Controls: 2
Outputs: 5
Delay embedding: 2


In [9]:
# Augmented state data
X = traj_augment_all.iloc[:, 0:n_state*delay_embedding]
X

,theta_offset_0,theta_dot_offset_0,x_offset_0,x_dot_offset_0,z_offset_0,z_dot_offset_0,theta_offset_-1,theta_dot_offset_-1,x_offset_-1,x_dot_offset_-1,z_offset_-1,z_dot_offset_-1
0,-0.141752,-16.592314,-17.582732,3.102667,1.086747,2.833017,0.142958,11.386702,-17.930353,4.479441,1.045827,-1.884903
1,-0.774046,3.583935,-17.041003,7.492268,1.623483,7.876772,-0.141752,-16.592314,-17.582732,3.102667,1.086747,2.833017
2,-0.718386,-2.368497,-16.071539,12.204198,2.660148,12.390169,-0.774046,3.583935,-17.041003,7.492268,1.623483,7.876772
3,-0.761949,1.183639,-14.647161,16.036803,4.029298,15.247972,-0.718386,-2.368497,-16.071539,12.204198,2.660148,12.390169
4,-0.763652,-0.764257,-12.950233,18.204426,5.620429,16.578333,-0.761949,1.183639,-14.647161,16.036803,4.029298,15.247972
...,...,...,...,...,...,...,...,...,...,...,...,...
260231,0.117936,1.437662,1.404614,0.325322,18.675687,1.488769,0.202850,-3.183932,1.355991,0.438261,18.497738,1.823627
260232,0.400448,3.951609,1.405008,0.250367,18.761771,0.804798,0.117936,1.437662,1.404614,0.325322,18.675687,1.488769
260233,0.645956,2.045812,1.451527,0.169752,18.800924,-0.029675,0.400448,3.951609,1.405008,0.250367,18.761771,0.804798
260234,0.785484,0.019339,1.439573,-0.002924,18.828193,-0.809622,0.645956,2.045812,1.451527,0.169752,18.800924,-0.029675


In [10]:
# Augmented control data
U = traj_augment_all.iloc[:, n_state*delay_embedding:n_state*delay_embedding+n_control*delay_embedding]
U

,j1_offset_0,j2_offset_0,j1_offset_-1,j2_offset_-1
0,8.049169,75.745171,-11.182978,59.740360
1,-2.380099,72.365653,8.049169,75.745171
2,1.415249,54.432573,-2.380099,72.365653
3,-0.803299,31.557822,1.415249,54.432573
4,0.490376,10.878398,-0.803299,31.557822
...,...,...,...,...
260231,1.002912,3.210493,1.887957,6.343482
260232,-0.742974,2.029763,1.002912,3.210493
260233,-0.819823,2.338930,-0.742974,2.029763
260234,0.028158,2.983043,-0.819823,2.338930


In [11]:
# Output data
Y = traj_augment_all.iloc[:, n_state*delay_embedding+n_control*delay_embedding:]
Y

,sensor_optic_flow_offset_0,sensor_theta_offset_0,sensor_theta_dot_offset_0,sensor_accel_x_offset_0,sensor_accel_z_offset_0
0,2.855536,-0.129697,-16.569165,10.081994,65.260270
1,4.561625,-0.796975,3.545528,51.148423,41.304389
2,4.540573,-0.715709,-2.382807,36.139968,30.897068
3,3.985252,-0.799967,1.188937,22.290788,12.499205
4,3.240890,-0.763251,-0.819222,7.519336,-1.976989
...,...,...,...,...,...
260231,0.000809,0.084429,1.433171,-0.344040,-6.620620
260232,-0.013789,0.369857,3.940717,-0.736389,-7.928407
260233,0.037798,0.706441,2.083396,-1.476087,-7.975956
260234,-0.025251,0.751732,-0.011484,-2.126294,-7.694932


### combine data

In [12]:
# Organize the data so we have the core data, and augmented (delay embedded) data
core_data = np.hstack((X.values[:, 0:n_state], U.values[:, 0:n_control]))
#aux_data = np.hstack((X.values[:, n_state:], U.values[:, n_control:]))
aux_data = core_data**-1
output_data = Y.values

In [13]:
all_data = {'X_core_train': core_data,
        'X_aux_train': aux_data,
        'y_train': output_data}

### Split up data into testing and training subsets

In [16]:
# split the data
data = keras_advanced_utility.prepare_train_test_split(all_data, test_size=0.2, random_state=42)

Data split complete:
  Training samples: 208188 (80%)
  Test samples: 52048 (20%)
  Core features: 8
  Aux features: 8
  Output dimension: 5


In [17]:
data.keys()

dict_keys(['X_core_train', 'X_core_test', 'X_aux_train', 'X_aux_test', 'y_train', 'y_test'])

# Build model architecture

In [18]:
# Define model architecture
core_architecture = [
    {'units': 64, 'activation': 'tanh'},
    {'units': 64, 'activation': 'tanh'}
]

aux_architecture = [
    {'units': 64, 'activation': 'tanh'}
]

combined_architecture = [
    {'units': 32, 'activation': 'linear'},
    {'units': output_data.shape[1]}  # Output layer
]

input_architecture = {'core_input_dim': data['X_core_train'].shape[1],
                      'aux_input_dim': data['X_aux_train'].shape[1]}


training_parameters = {'jacobian_weight': 0.1,
                       'sv_weight': 0.01}

In [20]:
# Build model
print("\nBuilding model...")
model, dropout_layer = keras_advanced_utility.build_auxiliary_dropout_model(
    core_input_dim=input_architecture['core_input_dim'],
    aux_input_dim=input_architecture['aux_input_dim'],
    core_architecture=core_architecture,
    aux_architecture=aux_architecture,
    combined_architecture=combined_architecture,
    jacobian_weight=training_parameters['jacobian_weight'],
    sv_weight=training_parameters['sv_weight']
)


Building model...


2025-10-16 12:46:34.233284: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [21]:
# Compile model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    loss='mse', 
    metrics=['mae']
)
model.summary()

Model: "jacobian_regularized_model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 core_features (InputLayer)  [(None, 8)]                  0         []                            
                                                                                                  
 aux_features (InputLayer)   [(None, 8)]                  0         []                            
                                                                                                  
 dense (Dense)               (None, 64)                   576       ['core_features[0][0]']       
                                                                                                  
 aux_dropout (Dropout)       (None, 8)                    0         ['aux_features[0][0]']        
                                                                         

In [22]:
# Train with curriculum learning
print("\nTraining with curriculum learning...")
history, curriculum_callback, X_core_val, X_aux_val, y_val = keras_advanced_utility.train_with_curriculum(
    model=model,
    dropout_layer=dropout_layer,
    X_core_train=data['X_core_train'],
    X_aux_train=data['X_aux_train'],
    y_train=data['y_train'],
    validation_split=0.2,
    epochs=200,
    batch_size=128,
    initial_dropout=0.5, # start with dropping out 20% of the auxilliary inputs
    final_dropout=0.99,  # end with dropping out 95% of the auxiliary inputs
    warmup_fraction=0.5,
    verbose=1
)


Training with curriculum learning...
Curriculum learning schedule:
  Total epochs: 200
  Warmup epochs: 100 (50% of training)
  Dropout: 0.50 → 0.99
  Final dropout reached at epoch 100
Epoch 1/200
1302/1302 [==============================] - 10s 7ms/step - loss: 10.5465 - mae: 1.6011 - jacobian_smoothness: 1.3555e-05 - sv_loss: 0.0824 - val_loss: 6.7992 - val_mae: 1.1126 - val_jacobian_smoothness: 0.0000e+00 - val_sv_loss: 0.0000e+00 - dropout_rate: 0.5000 - val_mae_with_aux: 1.1122 - val_mae_without_aux: 1.0787
Epoch 2/200
1302/1302 [==============================] - 8s 6ms/step - loss: 5.7892 - mae: 0.9685 - jacobian_smoothness: 8.2728e-05 - sv_loss: 2.8045e-04 - val_loss: 4.9971 - val_mae: 0.8663 - val_jacobian_smoothness: 0.0000e+00 - val_sv_loss: 0.0000e+00 - dropout_rate: 0.5049 - val_mae_with_aux: 0.8670 - val_mae_without_aux: 0.8382
Epoch 3/200
1302/1302 [==============================] - 8s 6ms/step - loss: 4.6685 - mae: 0.7844 - jacobian_smoothness: 1.8801e-04 - sv_loss: 9.

In [23]:
# Evaluate on validation and test sets
val_results = keras_advanced_utility.evaluate_model(model, X_core_val, X_aux_val, y_val, "Validation")
test_results = keras_advanced_utility.evaluate_model(model, data['X_core_test'], data['X_aux_test'], 
                               data['y_test'], "Test")

# Visualize results
print("\nGenerating visualizations...")
khf.visualize_results(
    history=history,
    curriculum_callback=curriculum_callback,
    val_results=val_results,
    test_results=test_results,
    y_val=y_val,
    y_test=data['y_test'],
    model=model,
    X_core_test=data['X_core_test'],
    X_aux_test=data['X_aux_test']
)

print("\n" + "="*60)
print("SUMMARY")
print("="*60)
print(f"Final dropout rate: {curriculum_callback.dropout_history[-1]:.2f}")
print(f"Mean Jacobian norm: {np.linalg.norm(np.random.randn(10, 5), axis=1).mean():.4f}")
print("\nTraining complete! Check 'curriculum_dropout_results.png' for visualizations.")


VALIDATION SET PERFORMANCE:
MAE without auxiliary features: 0.2672
MAE with auxiliary features:    0.2775
⚠ Without aux performs 3.72% better

TEST SET PERFORMANCE:
MAE without auxiliary features: 0.2722
MAE with auxiliary features:    0.2824
⚠ Without aux performs 3.62% better

Generating visualizations...


NameError: name 'khf' is not defined

In [ ]:
keras_advanced_utility.save_model_complete(
    model=model,
    filepath='smooth_measurements_model',  # No extension needed
    core_architecture=core_architecture,
    aux_architecture=aux_architecture,
    combined_architecture=combined_architecture,
    input_architecture=input_architecture,
    training_parameters=training_parameters
)